In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from pathlib import Path
from sklearn import cluster
from sklearn.preprocessing import StandardScaler
from skimage import io, color

In [7]:
FLATTENED_FILEPATH = "processed/flattened.csv"
PREDICTIONS_FILEPATH = "results/predictions.csv"

filenames = glob.glob("imgs/out*.png")
num_imgs = len(filenames)

processed_file = Path(FLATTENED_FILEPATH)
images = np.empty([num_imgs, 512, 512, 3])
if processed_file.is_file():
    print("Loading Flattened Images...")
    flattened_pd = pd.read_csv(processed_file, index_col=['frame', 'x', 'y'], skipinitialspace=True)
    flattened = flattened_pd.reset_index().values
    print("Finished Loading Flattened Images!")
else:
    print("Preprocessing Flattened Images...")   
    for i in range(num_imgs):
        images[i] = color.rgb2lab(io.imread(filenames[i]))
    flattened = np.empty([(num_imgs+4)*512*512, 6], dtype=np.int)
    for i in range(num_imgs):
        for x in range(512):
            for y in range(512):
                index = y + 512*(x + 512*i)
                '''if i >= num_imgs and i<num_imgs+2:
                    flattened[index] = np.concatenate([np.array([index, x, y], dtype=np.int), [97.14, -21.56, 94.48]])
                elif i >= (num_imgs + 2):
                    flattened[index] = np.concatenate([np.array([index, x, y], dtype=np.int), [67.29, 45.36, 47.49]])
                else:'''
                flattened[index] = np.concatenate([np.array([index, x, y], dtype=np.int), images[i, x, y]])
    print("Finished Flattening Images, working on extra images")
    for i in range(num_imgs, num_imgs + 2):
        for x in range(512):
            for y in range(512):
                index = y + 512*(x + 512*i)
                flattened[index] = np.concatenate([np.array([i, x, y], dtype=np.int), [97.14, -21.56, 94.48]])
    for i in range(num_imgs+2, num_imgs + 4):
        for x in range(512):
            for y in range(512):
                index = y + 512*(x + 512*i)
                flattened[index] = np.concatenate([np.array([i, x, y], dtype=np.int), [67.29, 45.36, 47.49]])
    os.makedirs(os.path.dirname(FLATTENED_FILEPATH), exist_ok=True)
    flattened_pd = pd.DataFrame(flattened, columns=['frame', 'x', 'y', 'red', 'green', 'blue'])
    flattened_pd = flattened_pd.set_index(['frame', 'x', 'y'], drop=True)
    flattened_pd.to_csv(FLATTENED_FILEPATH)
    print("Finished Preprocessing Flattened Images!")
    
flattened = flattened[:, 3:].astype(np.float)
print(flattened)

Preprocessing Flattened Images...


Finished Flattening Images, working on extra images


In [6]:
#scaler = StandardScaler()
#flattened = scaler.fit_transform(flattened)
clusterer = cluster.KMeans(8)

clusterer.fit(flattened)

colors = clusterer.cluster_centers_
#colors = scaler.inverse_transform(clusterer.cluster_centers_)

print("Selected colors:")
print(colors)

results = clusterer.predict(flattened)

print("Finished predictions!")
predictions = np.empty([flattened.shape[0], 6])
for i in range(num_imgs):
    for x in range(512):
        for y in range(512):
            index = y + 512*(x + 512*i)
            result = results[index]
            images[i, x, y] = colors[result]
            predictions[index] = np.concatenate([np.array([i, x, y], dtype=np.int), colors[result]])
    io.imsave(("results/res%d.png" % i), color.lab2rgb(images[i]))
print("Finished!")

os.makedirs(os.path.dirname(PREDICTIONS_FILEPATH), exist_ok=True)
predicted_pd = pd.DataFrame(predictions, columns=['frame', 'x', 'y', 'red', 'green', 'blue'])
predicted_pd = predicted_pd.set_index(['frame', 'x', 'y'], drop=True)
predicted_pd.to_csv(PREDICTIONS_FILEPATH)
print(predicted_pd)

Selected colors:
[[ 58.95251386 -22.34286911  -3.71547085]
 [ 67.08560844  44.804845    46.85142659]
 [ 63.82347732 -24.19768058  23.49752395]
 [ 97.         -21.          94.        ]
 [ 47.7477018  -17.30111488  -7.68350031]
 [ 70.67724193 -26.46348149  -4.78943882]
 [ 30.56908986  -6.9531388   -3.58867811]
 [ 51.56158711  17.42236862  20.7803117 ]]


Finished predictions!


/usr/local/lib/python3.5/dist-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


Finished!


                      red      green      blue
frame x   y                                   
0.0   0.0 0.0   30.569090  -6.953139 -3.588678
          1.0   30.569090  -6.953139 -3.588678
          2.0   30.569090  -6.953139 -3.588678
          3.0   30.569090  -6.953139 -3.588678
          4.0   30.569090  -6.953139 -3.588678
          5.0   30.569090  -6.953139 -3.588678
          6.0   30.569090  -6.953139 -3.588678
          7.0   30.569090  -6.953139 -3.588678
          8.0   30.569090  -6.953139 -3.588678
          9.0   30.569090  -6.953139 -3.588678
          10.0  30.569090  -6.953139 -3.588678
          11.0  30.569090  -6.953139 -3.588678
          12.0  30.569090  -6.953139 -3.588678
          13.0  30.569090  -6.953139 -3.588678
          14.0  30.569090  -6.953139 -3.588678
          15.0  47.747702 -17.301115 -7.683500
          16.0  47.747702 -17.301115 -7.683500
          17.0  47.747702 -17.301115 -7.683500
          18.0  47.747702 -17.301115 -7.683500
          19.